# Generation melodies with RNN-LSTM

In [1]:
from preprocess import PreProcessor
from hyperparameters import preprocessor_params
from training import Trainer
from rnn_lstm import RnnLstm
from hyperparameters import melody_generator_params
from melody_generation import MelodyGenerator
from music21 import midi


TRAIN = True

## Preprocessing stage

we need to take the data and process it

In [2]:
if TRAIN:
    pre = PreProcessor(**preprocessor_params)
    x, y = pre.process(remove_non_acceptable_durations=True, transpose=True)

Loading songs from "dataset/deutschl/test"...
Loaded 12 songs.
Removing non-acceptable notes & rests duration songs ...
Removed 0 songs.
Transposing songs to C maj / A min key...
Encoding the dataset into time series representation files...
Generating single file dataset & unique mapping...
processed_dataset/full_processed_dataset - single dataset file created!
processed_dataset/mapping.json - JSON mapping file created!


## Creating a RNN-LSTM model

more info about rnn lstm model and why we have chosen it

In [3]:
if TRAIN:
    rnn = RnnLstm(pre.outputs, [256], 0.001, pre.mapping_file)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, None, 18)]        0         
_________________________________________________________________
lstm (LSTM)                  (None, 256)               281600    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 18)                4626      
Total params: 286,226
Trainable params: 286,226
Non-trainable params: 0
_________________________________________________________________


## Training the model

some more info about training

In [4]:
if TRAIN:
    trainer = Trainer(rnn)
    trainer.fit(x, y, 64)
    trainer.save_model("trained_models")

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-09-03 18:42:06.957769: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-09-03 18:42:06.959207: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


40/40 [==============================] - 1s 23ms/step - loss: 1.9446 - accuracy: 0.5946
Epoch 2/50
40/40 [==============================] - 1s 23ms/step - loss: 1.1868 - accuracy: 0.7321
Epoch 3/50
40/40 [==============================] - 1s 23ms/step - loss: 1.1278 - accuracy: 0.7445
Epoch 4/50
40/40 [==============================] - 1s 23ms/step - loss: 1.0099 - accuracy: 0.7692
Epoch 5/50
19/40 [=============>................] - ETA: 0s - loss: 1.0787 - accuracy: 0.7431

KeyboardInterrupt: 

## Generating the melody

more info about generation

In [ ]:
if TRAIN:
    gen = MelodyGenerator(**melody_generator_params, model=trainer.model)
if not TRAIN:
    gen = MelodyGenerator(**melody_generator_params)
    gen.load_model("trained_models/erk_trained_model")
song = gen.generate_melody(500, 64, 0.85)
gen.save_melody(song)

mf = midi.MidiFile()
mf.open('mel.mid') # path='abc.midi'
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
s.show('midi')